In [74]:
%matplotlib inline
import torch
import numpy as np
import torch.nn as nn

In [75]:
'''生成数据集，跟之前一样'''
num_inputs = 2
num_examples = 1000
true_w = [2.1, -3.3]
true_b = 4.1
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1]+ true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

In [66]:
'''这一部分就是之前所写的data_iter函数，将数据集按小批处理'''
import torch.utils.data as Data

batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [67]:
'''定义线性网络模型，这里表示self.linear是nn库中的线性模型'''
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)
'''这一部分有多种写法，可以使用nn.Sequential'''

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [68]:
'''可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器'''
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[0.0468, 0.6850]], requires_grad=True)
Parameter containing:
tensor([0.2393], requires_grad=True)


In [69]:
'''
利用nn中自带的init函数初始化linear模型中weight和bias，normal_和constant_分别是
从正态分布和定值角度进行初始化
'''
from torch.nn import init

init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0) 

Parameter containing:
tensor([0.], requires_grad=True)

In [70]:
'''定义损失函数，使用nn库中MSELoss'''
loss = nn.MSELoss()

In [71]:
'''定义优化算法，使用SGD算法，输入网络中的参数和学习率'''
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [72]:
'''定义训练过程，按照批量对样本进行训练'''
optimizer.zero_grad()
num_epochs = 3
for epoch in range(1, num_epochs+1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output,y.view(output.size()))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss %f' %(epoch, l.item()))

epoch 1, loss 0.000273
epoch 2, loss 0.000038
epoch 3, loss 0.000055


In [73]:
'''输出最后结果'''
print(net.linear.weight)
print(net.linear.bias)


Parameter containing:
tensor([[ 2.1000, -3.3004]], requires_grad=True)
Parameter containing:
tensor([4.0998], requires_grad=True)
